<a href="https://colab.research.google.com/github/stefkong1982/netology.ru/blob/Master/Rekomendatelnye_sistemy/Gibridnye_RS/DZ_Kondratev_Gibridnye_RS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание к теме «Гибридные рекомендательные системы»

**Преподаватель:** Наталья Баданина, Иван Анисковец, Юлия Пономарева, Ярослав Сапронов, Егор Шишковец


Что нужно делать?

* Датасет ml-latest.
* Вспомнить подходы, которые мы разбирали.
* Выбрать понравившийся подход к гибридным системам.
* Написать свою.

# Основные таблицы в MovieLens

1. **movies**:
- **Описание**: Содержит информацию о фильмах.
- `movieId`: Уникальный идентификатор фильма.
- `title`: Название фильма.
- `genres`: Жанры фильма, обычно представлены в виде строки со списком жанров, разделенных символами `|` (например, "Action|Comedy").

2. **ratings**:
- **Описание**: Содержит оценки фильмов, выставленные пользователями.
- `userId`: Уникальный идентификатор пользователя.
- `movieId`: Уникальный идентификатор фильма (ссылается на таблицу `movies`).
- `rating`: Оценка (обычно от 0.5 до 5, с шагом 0.5).
- `timestamp`: Временная метка, указывающая, когда была оставлена оценка (обычно в формате Unix).


3. **tags** (не всегда присутствует):
- **Описание**: Содержит метки, оставленные пользователями на фильмы.
- `userId`: Уникальный идентификатор пользователя.
- `movieId`: Уникальный идентификатор фильма (ссылается на таблицу `movies`).
- `tag`: Текстовая метка, добавленная пользователем.
- `timestamp`: Временная метка, указывающая, когда была добавлена метка (обычно в формате Unix).

## Шаг 1: Установка и импорт необходимых библиотек

In [ ]:
# Установка библиотеки LightFM
!pip install lightfm

# Импорт необходимых библиотек
import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k

## Шаг 2: Загрузка данных

In [ ]:
# Загружаем данные из CSV-файлов в DataFrame
movies = pd.read_csv("https://raw.githubusercontent.com/stefkong1982/netology.ru/refs/heads/Master/Rekomendatelnye_sistemy/Rekomendacii_na_osnove_soderzhaniya/movies.csv")
ratings = pd.read_csv("https://raw.githubusercontent.com/stefkong1982/netology.ru/refs/heads/Master/Rekomendatelnye_sistemy/Rekomendacii_na_osnove_soderzhaniya/ratings.csv")
tags = pd.read_csv("https://raw.githubusercontent.com/stefkong1982/netology.ru/refs/heads/Master/Rekomendatelnye_sistemy/Rekomendacii_na_osnove_soderzhaniya/tags.csv")

## Шаг 3: Предварительный анализ данных

In [ ]:
tags

,userId,movieId,tag,timestamp
0,15,339,sandra 'boring' bullock,1138537770
1,15,1955,dentist,1193435061
2,15,7478,Cambodia,1170560997
3,15,32892,Russian,1170626366
4,15,34162,forgettable,1141391765
...,...,...,...,...
1291,660,135518,meaning of life,1436680885
1292,660,135518,philosophical,1436680885
1293,660,135518,sci-fi,1436680885
1294,663,260,action,1438398078


In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [ ]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary
9123,164977,The Gay Desperado (1936),Comedy


## Шаг 4: Подготовка признаков объектов (фильмов)

In [ ]:
# Разбиваем строку с жанрами на список жанров
movies['genres'] = movies['genres'].str.split('|')

In [ ]:
movies

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]
...,...,...,...
9120,162672,Mohenjo Daro (2016),"[Adventure, Drama, Romance]"
9121,163056,Shin Godzilla (2016),"[Action, Adventure, Fantasy, Sci-Fi]"
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,[Documentary]
9123,164977,The Gay Desperado (1936),[Comedy]


In [ ]:
# Группируем теги по movieId и собираем их в список
movie_tags = tags.groupby('movieId')['tag'].apply(list).to_dict()

In [ ]:
# Создаем словарь, где ключ - movieId, значение - список признаков (жанры + теги)
item_features_dict = {}
for _, row in movies.iterrows():
    movie_id = row['movieId']
    genres = row['genres']
    # Получаем теги для данного фильма, если они есть
    movie_tag_list = movie_tags.get(movie_id, [])
    # Объединяем жанры и теги
    features = genres + movie_tag_list
    item_features_dict[movie_id] = features

## Шаг 5: Создание объекта Dataset и подготовка данных для модели

In [ ]:
# Создаем объект Dataset
dataset = Dataset()

# Фитим Dataset на пользователях, фильмах и признаках фильмов
dataset.fit(users=ratings['userId'].unique(),
            items=movies['movieId'].unique(),
            item_features=set([feature for features in item_features_dict.values() for feature in features]))

## Шаг 6: Построение матрицы взаимодействий и матрицы признаков

In [ ]:
# Строим список (user_id, item_id) для взаимодействий
interactions_data = list(zip(ratings['userId'], ratings['movieId']))

# Создаем разреженную матрицу взаимодействий
(interactions, weights) = dataset.build_interactions(interactions_data)

In [ ]:
# Создаем матрицу признаков объектов
item_features = dataset.build_item_features(((item_id, features) for item_id, features in item_features_dict.items()))

## Шаг 7: Разделение данных на обучающую и тестовую выборки

In [ ]:
from lightfm.cross_validation import random_train_test_split

# Разбиваем данные на обучающую и тестовую выборки
train_interactions, test_interactions = random_train_test_split(interactions, test_percentage=0.2, random_state=42)

## Шаг 8: Обучение модели

In [ ]:
# Инициализируем модель LightFM с функцией потерь WARP
model = LightFM(loss='warp', random_state=42)

# Обучаем модель на обучающих взаимодействиях и признаках объектов
model.fit(train_interactions, item_features=item_features, epochs=10, num_threads=4)

## Шаг 9: Оценка модели

In [ ]:
# Вычисляем precision@k для обучающей и тестовой выборок
train_precision = precision_at_k(model, train_interactions, item_features=item_features, k=5).mean()
test_precision = precision_at_k(model, test_interactions, item_features=item_features, k=5).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.40, test 0.10.


- Вывод: `Precision: train 0.40, test 0.09.`
- Точность на обучающей выборке составила 0.40 (или 40%), что достаточно неплохо, так как это говорит о том, что в 40% случаев среди топ-5 рекомендуемых элементов есть релевантный элемент.
- Однако точность на тестовой выборке составила лишь 0.09 (или 9%). Это значительно ниже и может сигнализировать о  проблемах переобучением.
Модель, кажется, хорошо подстраивается под данные обучающей выборки, но не обобщает полученные знания на новые данные, что приводит к низкой точности на тестовой выборке.